# Исследование объявлений о продаже квартир

---

## Описание проекта

В вашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность.

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма.

---

## План работы

### 1. Изучить входные данные;
### 2. Выполнить предобработку данных;
### 3. Добавить новые столбцы;
### 4. Провести исследовательский анализ данных;
### 5. Сделать выводы.

---

## Информация о входных данных

В таблице 22 столбца:

**airports_nearest** — расстояние до ближайшего аэропорта в метрах (м)<br>
**balcony** — число балконов<br>
**ceiling_height** — высота потолков (м)<br>
**cityCenters_nearest** — расстояние до центра города (м)<br>
**days_exposition** — сколько дней было размещено объявление (от публикации до снятия)<br>
**first_day_exposition** — дата публикации<br>
**floor** — этаж<br>
**floors_total** — всего этажей в доме<br>
**is_apartment** — апартаменты (булев тип)<br>
**kitchen_area** — площадь кухни в квадратных метрах (м²)<br>
**last_price** — цена на момент снятия с публикации<br>
**living_area** — жилая площадь в квадратных метрах(м²)<br>
**locality_name** — название населённого пункта<br>
**open_plan** — свободная планировка (булев тип)<br>
**parks_around3000** — число парков в радиусе 3 км<br>
**parks_nearest** — расстояние до ближайшего парка (м)<br>
**ponds_around3000** — число водоёмов в радиусе 3 км<br>
**ponds_nearest** — расстояние до ближайшего водоёма (м)<br>
**rooms** — число комнат<br>
**studio** — квартира-студия (булев тип)<br>
**total_area** — площадь квартиры в квадратных метрах (м²)<br>
**total_images** — число фотографий квартиры в объявлении<br>

Пояснение: апартаменты — это нежилые помещения, не относящиеся к жилому фонду, но имеющие необходимые условия для проживания.<br>

---

### Оглавление

* [1. Открываем файл с данными и изучаем общую информацию](#one)
* [2. Предобработка данных](#two)
* [3. Добавляем новые столбцы](#six)
* [4. Исследовательский анализ данных и выполнение инструкций](#seven)
*     [Изучение параметров](#eight)
*    [Изучение время продажи квартиры](#nine)
*     [Какие факторы больше всего влияют на стоимость квартиры](#ten)
*   [10 населённых пунктов с наибольшим числом объявлений](#eleven)
* [Изучение предложения квартир](#twelve)
* [Cегмент квартир в центре](#thirteen)
* [5. Выводы](#fourtheen)

---

## Шаг 1. Открываем файл с данными и изучаем общую информацию. <a class="anchor" id="one"></a>

Откроем файл с данными. Посмотрим на первые пятнадцать строк методом head() и изучим столбцы методом info().

In [ ]:
import pandas as pd
from IPython.display import display
import matplotlib as plt

In [ ]:
flats = pd.read_csv('datasets/2_real_estate_data.csv', sep='\t')

In [ ]:
display(flats.head(15))

In [ ]:
flats.info()

In [ ]:
flats.iloc[:15, :15]

## Вывод

Открыли файл и увидели 22 столбца.
Обнаружили пропущенные значения во многих столбцах: в столбце is_apartment пропущенные значения заменим на 0, т.к. это булево значениеи логично, что это не апартаменты, а квартира.
Аналогично в столбце balcony, т.к. пропущенное значение говорит о том, что балкона нет.
В столбце ceiling_height и living_area, предположительно, владельцы не внесли значение т.к. не знают этот показатель, будем искать логику по заполнению пропущенных значений.
В столбце days_exposition пропущенные значения говорят о том, что недвижимость еще не продана.
Также автоматически заполненные данные по паркам, водоемам и аэропортам не заполнены по некоторым поселкам, поэтому заполним средним значением по данному поселку, если такие данные имеются, либо выделим их в отдельную группы "Провинция".

Обнаружен артефакт - площадь кухни больше площади квартиры.

В следующем шаге заменим кол-во балконов и данные, полученные от карт на целочисленные.
И заменим тип данных в дате объявления.

---

## Шаг 2. Предобработка данных <a class="anchor" id="two"></a>

### Замена пропущенных значений <a class="anchor" id="three"></a>

Проверим наш датафрейм на дубликаты

In [ ]:
flats.duplicated().sum()

Дубликатов не обнаружено.

Последовательно по каждому столбцу будем проверять на дубликаты и преобразовывать данные.

In [ ]:
flats['total_images'].unique()

За квартиру никто копейками не платит, поэтому переведем цену в целочисленный тип данных и посмотрим на значения.

In [ ]:
flats['last_price'] = pd.to_numeric(flats['last_price'], errors='raise', downcast='integer')
flats['last_price'].describe()

Интересная цена получилась от 12 к и интересно много таких отчаянных?) Посмотрим на самые низкие и высокие цены, отсортировав по столбцу цены.

In [ ]:
flats = flats.sort_values(by='last_price')

flats.head(50)

In [ ]:
flats.tail(15)

Пейнтхаусы за огромные суммы еще ок, но недвижимость за копейки это нонсенс. После преобразований столбцов обязательно вернемся к обработке таких цен.

In [ ]:
flats['last_price'].describe()

In [ ]:
flats['total_area'].describe()

Сперва смутила площадь в 12 м, но оказалось такие есть)
Преобразуем дату, время нас не интересует, поэтому оставим только дату и год.

In [ ]:
flats['first_day_exposition'] = pd.to_datetime(flats['first_day_exposition'], format='%Y-%m-%d')
flats.info()

In [ ]:
flats['rooms'].describe()

In [ ]:
flats['ceiling_height'].describe()

In [ ]:
flats['ceiling_height'].dropna().sort_values().tail(50)

In [ ]:
flats['ceiling_height'].dropna().sort_values().head(50)

В таблице появилась недвижимость для хоббитов с высотой 1 м. Будь мы во Вселенной Толкиена оставили бы все как есть, но в наших реалиях минимальная высота потолков 2,4 метра. 
По максимальной высоте потолков ограничений нет. Могут быть квартиры из которых можно сделать два этажа. При этом свыше 6 метров предлагаю откинуть. Значения от 24 метров до 32 были внесены неправильно, поэтому поделим их на порядок и 100 откинем.
Пустые значения можно было бы заменить на среднее значение в данном районе, т.к. архитектура в районах похожа, но в нашем случае дан лишь город.
Такое огромное значение пропусков в ввиду того, что данное поле необязательное.

Пустые значения заменим медианой.

In [ ]:
#  Избавляемся от некорректных значений
def ceiling_height(row):
    if row is not None:
        if row < 2.4:
            return None
        elif 6 < row < 24:
            return None
        elif 24 <= row < 100:
            return row/10
        elif row >= 100:
            return None
        return row
flats['ceiling_height'] = flats['ceiling_height'].apply(ceiling_height)

# Замена на медиану

median_height = flats['ceiling_height'].median()
flats.loc[flats['ceiling_height'].isna(), 'ceiling_height'] = median_height

In [ ]:
flats.info()

Заменим пропущенные значения в количестве этажей медианой, сгруппировав данные по этажу.

In [ ]:
# Функция принимает список столбцов и показывает пропуски до изменений

def null_before(column, data=flats):
    print(f'Пропуски в столбце {column} до:', data[column].isna().sum())
        
# Функция принимает список столбцов и показывает пропуски до изменений

def null_after(column, data=flats):
    print(f'Пропуски в столбце {column} после:', data[column].isna().sum())
        
# Функция принимает список столбцов и дает название медианам

def median_column(column, grouped_column, data):
    column_median = (data.groupby([grouped_column])).agg({column:'median'})\
    .rename(columns = {column: 'median_name'})
    data = data.merge(column_median, on = [grouped_column])
    data.loc[data[column].isna(), column] = data.loc[data[column].isna(), 'median_name']
    data[column] = data[column].fillna(data[column].median())
    return data
                     
# Функция принимает датасет и столбец и удаляет столбец
                     
def drop_column(column, data=flats):
    return data.drop([column], axis=1)
                     
# Функция принимает столбец А, по которому группирует, столбец Б, по которому ищет пропуски до, вычисляет медиану,\
# добавляет столбец В с медианой к исходному фрейму, если столбец В оказался пустым, то заполняет его медианой\
# столбца Б по всему датафрейму, заполняет пропуски, вычисляет пропуски после и удаляет лишние столбцы с медианой
                     

def clear_null(column, grouped_column, data=flats):
    null_before(column, data)
    data = median_column(column, grouped_column, data)
    null_after(column, data)
    data = drop_column('median_name', data)
    return data

# Функция принимает список столбцов и столбец, по которому будет группировать и заменяет значения на медианные
def clear_group(group_columns, grouped_column, data=flats):
    for group_column in group_columns:
        data = clear_null(group_column, grouped_column, data)
    return data

flats = clear_null('floors_total', 'floor')

In [ ]:
flats['living_area'].describe()

In [ ]:
flats.query('living_area == kitchen_area')

Жилая площадь в 2 метра это как раз, чтобы уместилась кровать) Ранее мы обнаружили, что есть квартиры, где площадь кухни больше жилой площади. Вероятно по ошибке перепутали местами эти значения. Заменим их обратно.

In [ ]:
# Если площадь кухни больше жилой площади, заменим ее на плодащь жилой площади
def change_kitchen_living(row):
    live = row['living_area']
    kitchen = row['kitchen_area']
    area = row['total_area']
    if kitchen is not None and live is not None:
        if live < kitchen:
            return kitchen
        elif live == kitchen:
            if area - live > live:
                return area - live 
    return live
flats['true_living_area'] = flats.apply(change_kitchen_living, axis=1)
def change_living_kitchen(row):
    live = row['living_area']
    kitchen = row['kitchen_area']
    area = row['total_area']
    if kitchen is not None and live is not None:
        if kitchen > live:
            return live
        elif kitchen == live:
            if area - live < live:
                return area - live
    return kitchen
flats['true_living_area'] = flats.apply(change_kitchen_living, axis=1)
flats['kitchen_area'] = flats.apply(change_living_kitchen, axis=1)
flats['living_area'] = flats['true_living_area']

flats = drop_column('true_living_area', flats)

Избавимся от пропусков в столбцах жилая площадь и площадь кухни медианой, сгруппировав их по площади помещения.
Площадь помещения поделим на группы разделив их квартилями.

In [ ]:
# Посчитаем квартили и медиану
area_median_1 = flats['total_area'].quantile(0.25)
area_median_2 = flats['total_area'].quantile(0.5)
area_median_3 = flats['total_area'].quantile(0.75)

# Добавим категорию 1, 2, 3
def area_category(row):
    area = row['total_area']
    if area <= area_median_1:
        return 1
    elif area <= area_median_2:
        return 2
    elif area <= area_median_3:
        return 3
    return 4

# Добавим новый столбец area_category к датафрейму
flats['area_category'] = flats.apply(area_category, axis=1)
print('Первая категория имеет площадь от {:.1f} до {:.1f} включительно'\
                      .format(flats['total_area'].min(), area_median_1))
print('Вторая категория имеет площадь от {:.1f} до {:.1f} включительно'\
                      .format(area_median_1, area_median_2))
print('Третья категория имеет площадь от {:.1f} до {:.1f} включительно'\
                      .format(area_median_2, area_median_3))
print('Четвертая категория имеет площадь от {:.1f} до {:.1f} включительно'\
                      .format(area_median_3, flats['total_area'].max())) 

# Посчитаем медиану для каждой группы

Заполним пропуски функцией clear_group()

In [ ]:
flats = clear_group(['living_area', 'kitchen_area'], 'area_category', flats)

In [ ]:
flats.info()

От пропусков избавились. Посмотрим на другие столбцы.

In [ ]:
flats['floor'].unique()

In [ ]:
flats['is_apartment'].unique()

Заменим пустые значения на False, т.к. пропущенное значение означает, что это не апартамент

In [ ]:
# Пропуски до 
null_before('is_apartment', flats)

# Удалим пропуски методом fillna
flats['is_apartment'] = flats['is_apartment'].fillna(False)

# Пропуски после
null_after('is_apartment', flats)

Как мы видели ранее у нас есть пропущенные значения в поле кол-во балконов. Это говорит, что балконы отсутствуют. Заменим их на 0.

In [ ]:
flats['balcony'].unique()

In [ ]:
# Пропуски до 
null_before('balcony', flats)

# Удалим пропуски методом fillna
flats['balcony'] = flats['balcony'].fillna(0)

# Пропуски после
null_after('balcony', flats)

# Заменим тип данных на целочисленный
flats['balcony'] = flats['balcony'].astype(int)

В названии населенного пункта были пропуски, их крайне сложно заменить на что либо. Т.к. их мало просто удалим строки с данными значениями.

In [ ]:
# Подсчет пропущенных значениях в районе
null_before('locality_name', flats)

# Удалим пропущенные значения в районе
flats = flats.dropna(subset=['locality_name'])

# Подсчет пропущенных значениях в районе
null_after('locality_name', flats)

flats['locality_name'].unique()

Увидели разное написание поселка. Поэтому заменим все посЁлки на поселки и приведем к нижнему регистру.

In [ ]:
# Приведем к нижнему регистру
flats['locality_name'] = flats['locality_name'].str.lower()


# Избавимся от поселков
def poselok(row):
    sense = row['locality_name']
    a = ''
    b = 0
    for i in sense.split():
        if i == 'посёлок':
            i = 'поселок'
        if b > 0:
            a += ' '
            a += i
        else:
            a += i
            b += 1
            
    return a

flats['locality_name'] = flats.apply(poselok, axis=1)
flats['locality_name'].unique()

В столбцах полученных от карт много пропусков. Заменим их на медиану населенного пункта, если у населенного пункта нет медианы, то заменим на медиану датасета.

In [ ]:
# Для удобства заменим имя столбца cityCenters_nearest
flats.columns = flats.columns.str.lower()

# Заполним пропуски с помощью функции clear_group
flats = clear_group(['airports_nearest', 'citycenters_nearest', 'parks_around3000',\
                     'parks_nearest', 'ponds_around3000', 'ponds_nearest'], 'locality_name', flats)

In [ ]:
flats.info()

Ранее мы заметили, что в столбце days_exposition имеются пропуски. Давайте посмотрим на них.

In [ ]:
# задаем новый столбец, к-й определяет пропущена дата или нет
flats['days_exposition_missing'] = flats['days_exposition'].isna().astype(int)

# суммируем и строим графики
flats.groupby('first_day_exposition').agg({'days_exposition_missing':'sum'})\
    .plot(title='Зависимость пропущенных значений в столбце days_exposition от first_day_exposition')

Число пропусков растет к концу наблюдаемого периода. Видимо, пропуски означают незакрытые объявления. Будем считать, что дата выгрузки данных равна последнему дню наблюдений.

In [ ]:
null_before('days_exposition', flats)
max_dt = flats['first_day_exposition'].max()

# заполним пропуски разностью first_day_exposition и max_dt в днях
flats.loc[flats['days_exposition'].isna(), 
         'days_exposition'] = (max_dt - flats.loc[flats['days_exposition']\
                                    .isna(), 'first_day_exposition']).dt.days
null_after('days_exposition', flats)

Преобразуем данные из карт к целочисленному типу и удалим промежуточный столбец

In [ ]:
# Выполним преобразование типов
convert_dict = {'airports_nearest': int,
               'citycenters_nearest': int,
               'parks_around3000': int,
               'parks_nearest': int,
                'ponds_around3000': int,
               'ponds_nearest': int,
               'days_exposition': int}

flats = flats.astype(convert_dict)

# Удалим ненужный столбец
#flats = drop_column('days_exposition_missing', flats)

flats.info()

### Выводы:

В данном шаге изменили типы данных с следующих столбцов:
*Привели к целочисленному типу*
**airports_nearest** — расстояние до ближайшего аэропорта в метрах (м)<br>
**balcony** — число балконов<br>
**cityCenters_nearest** — расстояние до центра города (м)<br>
**days_exposition** — сколько дней было размещено объявление (от публикации до снятия)<br>
**floor** — этаж<br>
**floors_total** — всего этажей в доме<br>
**last_price** — цена на момент снятия с публикации<br>
**parks_around3000** — число парков в радиусе 3 км<br>
**parks_nearest** — расстояние до ближайшего парка (м)<br>
**ponds_around3000** — число водоёмов в радиусе 3 км<br>
**ponds_nearest** — расстояние до ближайшего водоёма (м)<br>

*Привели к дате*

**first_day_exposition** — дата публикации<br>

Избавились от пропусков в следующих столбцах:<br>
**airports_nearest** — расстояние до ближайшего аэропорта в метрах (м)<br>
**balcony** — число балконов<br>
**cityCenters_nearest** — расстояние до центра города (м)<br>
**days_exposition** — сколько дней было размещено объявление (от публикации до снятия)<br>
**floors_total** — всего этажей в доме<br>
**parks_around3000** — число парков в радиусе 3 км<br>
**parks_nearest** — расстояние до ближайшего парка (м)<br>
**ponds_around3000** — число водоёмов в радиусе 3 км<br>
**ponds_nearest** — расстояние до ближайшего водоёма (м)<br>
**is_apartment** — апартаменты (булев тип)<br>
**kitchen_area** — площадь кухни в квадратных метрах (м²)<br>
**living_area** — жилая площадь в квадратных метрах(м²)<br>
**ceiling_height** — высота потолков (м)<br>

Удалили строки с пропущенным значением в столбце **locality_name** и избавились от дубликатов, приведя к нижнему регистру и изменив Ё на Е.

---

## Шаг 3. Добавление новых столбцов в таблицу <a class="anchor" id="six"></a>

Вычислим цену квадратного метра



In [ ]:
flats['price_per_m'] = flats['last_price'] / flats['total_area']

In [ ]:
flats['price_per_m'] = flats['price_per_m'].astype(int)

In [ ]:
flats.head()

День недели, месяц и год публикации объявления

In [ ]:
flats['day_of_week'] = flats['first_day_exposition'].dt.dayofweek
flats['month'] = flats['first_day_exposition'].dt.month
flats['year'] = flats['first_day_exposition'].dt.year
flats.head(10)

Вычислим этаж квартиры; варианты — первый, последний, другой;

In [ ]:
# Функция принимает строку в таблице и принимает тип этажа
def floor_kind(row):
    floor = row['floor']
    total_floor = row['floors_total']
    if floor == 1:
        return 'первый'
    elif floor == total_floor:
        return 'последний'
    return 'другой'
flats['floor_kind'] = flats.apply(floor_kind, axis=1)

flats['floor_kind'].value_counts()

Соотношение жилой и общей площади, а также отношение площади кухни к общей.

In [ ]:
# Соотношений жилой площади к общей
flats['living_total'] = flats['living_area'] / flats['total_area']

# Соотношение кухни к общей площади
flats['kitchen_total'] = flats['kitchen_area'] / flats['total_area']

flats.head()

flats.info()

### Выводы

1. В таблицу добавлен новый столбец с ценой за квадратный метр.
2. Добавлены день недели, месяц и год публикации объявления;
3. Добавлена категория по этажам: Первый, последний, другой;
4. Добавлены соотношения жилой площади и кухни к общей площади недвижимости.

---

## Шаг 4. Проведем исследовательский анализ данных и выполним инструкции: <a class="anchor" id="seven"></a>

### Изучение параметров<a class="anchor" id="eight"></a>

Мы изучим следующие параметры: площадь, цена, число комнат, высота потолков. И построим гистограммы для каждого параметра.

In [ ]:
# Импортируем библиотеку для построения графиков
import matplotlib.pyplot as plt

Для каждого параметра построим гистограмму распределения в пределах 1,5 межквартильных расстояния от квартилей и тем самым избавимся от редких значений. И посмотрим на параметры методом describe().

In [ ]:
parameters = ['total_area', 'last_price', 'rooms', 'ceiling_height']
for parameter in parameters:
    q1 = flats[parameter].quantile(0.25)
    q3 = flats[parameter].quantile(0.75)
    iqr = q3 - q1
    
    flats[parameter].plot(kind = 'hist', bins = 50, range = \
                          (q1 - 1.5 * iqr, q3 + 1.5 * iqr), grid=True, figsize = (10,7), title = parameter)
    display(flats[parameter].describe())
    plt.show()
    

### Выводы

Площадь недвижимости:
1. Распределение от 12 до 900 метров;
2. Распределение унимодально;
3. Среднее значение равно 60 кв м;
4. Медианное значение равно 52 кв м.

Стоимость недвижимости:
1. Распределение от 12 к до 763 млн;
2. Распределение унимодально;
3. Средняя цена 6,5 млн;
4. Медианное значение цены 4,65 млн.

Комнаты в недвижимости:
1. Распределение от 0 до 19;
2. Распределение унимодально;
3. Среднее число комнат совпадает с медианным и равно 2.

Выоста потолка:
1. Распределение от 2,4 до 6 метров;
2. Распределение унимодально;
3. Среднее значение примерно равно медианному 2,7 метров.

---

### Изучение времени продажи квартиры <a class="anchor" id="nine"></a>

In [ ]:
q1 = flats['days_exposition'].quantile(0.25)
q3 = flats['days_exposition'].quantile(0.75)
iqr = q3 - q1
    
flats['days_exposition'].plot(kind = 'hist', bins = 50, range = (1, q3 + 1.5 * iqr), \
                              grid=True, figsize = (10,7), title = 'days_exposition')
display(flats['days_exposition'].describe())

### Выводы

1. Получили геометрическое распределение времени продажи недвижимости.
2. Распределение от 0 до 1618 дней. Кому-то пришлось ждать 4,5 года...
3. Среднее время продажи полгода;
4. Медианное время продажи 94 дня.
5. Долгим я бы посчитал свыше 200 дней, а необычно долгим более года.
6. От редких значений избавились путем ограничения по оси от 1 до q3(квартиль 75%) + 1,5 iqr (межквартильное расстояние)

---

### Какие факторы больше всего влияют на стоимость квартиры <a class="anchor" id="ten"></a>

Давайте выполним это построением попарных точечных диаграмм диаграмм для столбцов датафрейма.
Изучим, зависит ли цена от площади, числа комнат, удалённости от центра. Изучим зависимость цены от того, на каком этаже расположена квартира: первом, последнем или другом. Также изучите зависимость от даты размещения: дня недели, месяца и года.

In [ ]:
# Присвоим категории этажа числовое значение
def category_floor(row):
    floor = row['floor_kind']
    if floor == 'первый':
        return 1
    elif floor == 'другой':
        return 2
    return 3

flats['category_floor'] = flats.apply(category_floor, axis=1)


# Построим графики зависимости и посчитаем коэффициент корреляции 

parameters = ['total_area', 'rooms', 'citycenters_nearest', 'category_floor', \
                                              'day_of_week', 'month', 'year']
q11 = flats['last_price'].quantile(0.25)
q33 = flats['last_price'].quantile(0.75)
iqr2 = q33 - q11
for parameter in parameters:
    q1 = flats[parameter].quantile(0.25)
    q3 = flats[parameter].quantile(0.75)
    iqr = q3 - q1

    
    flats.plot(kind = 'scatter', y='last_price', x=parameter, title=parameter, \
               ylim = (q11 - 1.5 * iqr2, q33 + 1.5 * iqr2), xlim = (q1 - 1.5 * iqr, q3 + 1.5 * iqr))
    display(flats[parameter].corr(flats['last_price']))
    plt.show()

### Выводы

1. Стоимость недвижимости коррелирует с площадью квартиры;
2. Немного слабже стоимость коррелирует с количеством комнат;
3. Еще слабже отрицательная связь между стоимостью и удаленностью т.е. чем больше удаленность тем меньше цена.
4. Остальные величины не коррелируют со стоимостью недвижимости.

---

### 10 населённых пунктов с наибольшим числом объявлений <a class="anchor" id="eleven"></a>

Посчитаем среднюю цену квадратного метра в этих населённых пунктах. Выделим среди них населённые пункты с самой высокой и низкой стоимостью жилья. Эти данные можно найти по имени в столбце 'locality_name'.

In [ ]:
ten_locality = flats.pivot_table(index = ['locality_name'], values = ['price_per_m', 'last_price'],\
                            aggfunc = {'price_per_m':['count', 'mean'], 'last_price':['min','max']})
ten_locality.columns = ['максимальная цена', 'минимальная цена', 'количество', 'средняя цена за кв метр']
ten_locality = ten_locality.sort_values(by='количество', ascending=False)
display(ten_locality.head(10).style.format({'средняя цена за кв метр':'{:.0f}'}))
display(ten_locality.head(10).sort_values(by='средняя цена за кв метр', ascending=False)\
                            .style.format({'средняя цена за кв метр':'{:.0f}'}))

### Выводы

1. Получили десять населенных пунктов с наибольшим количеством объявлений;
2. Средняя цена за квадратный метр самая высокая в Санкт-Петербурге и Пушкине, немного ниже в деревне Кудрово и поселке Мурино.
3. Максимальная цена недвижимости в Санкт-Петербурге. 
4. Из-за некорректного выброса в 12 к в Санкт-Петербурге самая низкая цена, если его отбросить, то в Выборге.

---

### Изучение предложения квартир <a class="anchor" id="twelve"></a>

Изучим предложения квартир: для каждой квартиры есть информация о расстоянии до центра. Выделим квартиры в Санкт-Петербурге ('locality_name'). Наша задача — выяснить, какая область входит в центр. Создадим столбец с расстоянием до центра в километрах: округлим до целых значений. После этого посчитаем среднюю цену для каждого километра. Построим график: он должен показывать, как цена зависит от удалённости от центра. Определим границу, где график сильно меняется — это и будет центральная зона.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

flats['city_center_km'] = flats['citycenters_nearest'] // 1000
centr_flats = flats.query('locality_name == "санкт-петербург"')
centr_flats_1 = centr_flats.groupby('city_center_km').agg({'price_per_m': 'mean'}) \
                                .rename(columns = {'price_per_m': 'Средняя цена м2'})
centr_flats_1['Средняя цена м2'].plot(title='Зависимость цены за квадратный метр от удаленности от центра', \
    figsize=(15,4))
plt.show()

### Выводы

1. Мы выделили всю недвижимость в Санкт-Петербурге;
2. Мы построили график зависимости цены за квадратный метр от удаленности от центра;
3. Резкое снижение цена в области 7,5 км - это и будет центр города;
4. На расстоянии 27 км увидели выброс.

---

### Cегмент квартир в центре <a class="anchor" id="thirteen"></a>

Выделим сегмент квартир в центре. Проанализируем эту территорию и изучим следующие параметры: площадь, цена, число комнат, высота потолков. Также выделим факторы, которые влияют на стоимость квартиры (число комнат, этаж, удалённость от центра, дата размещения объявления). Сделаем выводы. Отличаются ли они от общих выводов по всему городу?

In [ ]:
# Посмотрим на распределение по площади, цене, числе комнат и высоте потолков.
centr_flats = centr_flats.query('city_center_km <= 7.5')
parameters = ['total_area', 'last_price', 'rooms', 'ceiling_height']
for parameter in parameters:
    q1 = centr_flats[parameter].quantile(0.25)
    q3 = centr_flats[parameter].quantile(0.75)
    iqr = q3 - q1
    
    centr_flats[parameter].plot(kind = 'hist', bins = 50, \
        range = (q1 - 1.5 * iqr, q3 + 1.5 * iqr), grid=True, figsize = (10,7), title = parameter)
    display(centr_flats[parameter].describe())
    plt.show()

Площадь недвижимости:
1. Распределение от 12 до 900 метров;
2. Распределение унимодально;
3. Среднее значение равно 92 кв м;
4. Медианное значение равно 78 кв м.

Стоимость недвижимости:
1. Распределение от 1.6 м до 763 млн;
2. Распределение унимодально;
3. Средняя цена 14,7 млн;
4. Медианное значение цены 9 млн.

Комнаты в недвижимости:
1. Распределение от 0 до 19;
2. Распределение унимодально;
3. Среднее число комнат совпадает с медианным и равно 3.

Выоста потолка:
1. Распределение от 2,4 до 6 метров;
2. Распределение геометрическое;
3. Среднее значение примерно равно 2,9 метров;
4. Медианноке - 2,7 метров.

In [ ]:
# Построим графики зависимости и посчитаем коэффициент корреляции 

parameters = ['total_area', 'rooms', 'citycenters_nearest', 'category_floor', 'day_of_week', 'month', 'year']
q11 = centr_flats['last_price'].quantile(0.25)
q33 = centr_flats['last_price'].quantile(0.75)
iqr2 = q33 - q11
for parameter in parameters:
    q1 = centr_flats[parameter].quantile(0.25)
    q3 = centr_flats[parameter].quantile(0.75)
    iqr = q3 - q1

    
    centr_flats.plot(kind = 'scatter', y='last_price', x=parameter, title=parameter, \
        ylim = (q11 - 1.5 * iqr2, q33 + 1.5 * iqr2), xlim = (q1 - 1.5 * iqr, q3 + 1.5 * iqr))
    display(centr_flats[parameter].corr(centr_flats['last_price']))
    plt.show()

1. Стоимость недвижимости коррелирует с площадью квартиры;
2. Немного слабже стоимость коррелирует с количеством комнат;
3. Еще слабже отрицательная связь между стоимостью и удаленностью т.е. чем больше удаленность тем меньше цена.
4. Остальные величины не коррелируют со стоимостью недвижимости.

### Выводы

**Общая площадь**

1. Средняя площадь изменилась с 60 квадратных метров на 92;
2. Медианное значение площади изменилось с 52 до 78 квадратных метров.
3. В остальном поведение похоже.

**Стоимость недвижимости**

1. Минимальная стоимость увеличилась до 1.6 млн;
2. Средняя цена изменилась с 6,5 до 14,7 млн;
3. Медианное значение стоимости изменилось с 4,65 до 9 млн;
4. Поведение осталось похоже.

**Комнаты в недвижимости**

1. Поведение похоже;
2. Среднее и медианное значение увеличилось с 2 до 3. Видимо в центре люди чаще продают недвижимость с большим количеством комнат;

**Высота потолка**

1. Среднее значение увеличилось до 2.9 метров. В центре люди чаще продают недвижимость с более высокими потолками.

**Зависимость стоимости недвижимости от общей площади**

1. Стоимость недвижимости коррелирует с площадью квартиры аналогично;

**Зависимость стоимости недвижимости от количества комнат**

Зависимость немного слабее, в центре и в городе зависимость аналогична.

**Зависимость стоимости недвижимости и удаленностью**

В центре зависимости совсем отсутствует, при этом в городе наблюдается обратная корреляция.

**Остальные величины не коррелируют со стоимостью недвижимости**

---

## Шаг 5. Общий вывод <a class="anchor" id="fourtheen"></a>

Краткий обзор проведенной работы.

Для исследование объявлений о продаже квартир мы открыли файл Заказчика и изучили информацию. Архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет содержит 22 столбца и 23699 строк. Обнаружили артефакты в виде площади кухни большей жилой площади. 

Отработали пропущенные значения и сделали выводы в втором шаге.

Изменили тип данных на целочисленный и перевели в формат времени.

Удалили 49 строк из датасета.

**В третьем шаге выполнили следующее:**

1. В таблицу добавили новый столбец с ценой за квадратный метр.
2. Добавили день недели, месяц и год публикации объявления;
3. Добавили категория по этажам: Первый, последний, другой;
4. Добавили соотношения жилой площади и кухни к общей площади недвижимости.

**Главные выводы**

1. Стоимость недвижимости сильно коррелирует с площадью недвижимости;
2. В центре Санкт-Петербурга общая площадь недвижимости больше, чем в пригороде;
3. Стоимость недвижимости зависит от близости к центру, чем ближе недвижимость к центру, тем выше стоимость. В самом центре отсутствует;
4. Больше всего объявлений с 1, 2 и 3 комнатной недвижимостью. Чем больше комнат, тем дороже недвижимость.
5. Остальные характеристики не сильно влияют на стоимость недвижимости.

Рекомендации:
1. Для более точного анализа рекомендуем добавить поле адрес в данные.

---

### Чек-лист готовности проекта

- [x]  открыт файл
- [x]  файлы изучены (выведены первые строки, метод info())
- [x]  определены пропущенные значения
- [x]  заполнены пропущенные значения
- [x]  есть пояснение, какие пропущенные значения обнаружены
- [x]  изменены типы данных
- [x]  есть пояснение, в каких столбцах изменены типы и почему
- [x]  посчитано и добавлено в таблицу: цена квадратного метра
- [x]  посчитано и добавлено в таблицу: день недели, месяц и год публикации объявления
- [x]  посчитано и добавлено в таблицу: этаж квартиры; варианты — первый, последний, другой
- [x]  посчитано и добавлено в таблицу: соотношение жилой и общей площади, а также отношение площади кухни к общей
- [x]  изучены следующие параметры: площадь, цена, число комнат, высота потолков
- [x]  построены гистограммы для каждого параметра
- [x]  выполнено задание: "Изучите время продажи квартиры. Постройте гистограмму. Посчитайте среднее и медиану. Опишите, сколько обычно занимает продажа. Когда можно считать, что продажи прошли очень быстро, а когда необычно долго?"
- [x]  выполнено задание: "Уберите редкие и выбивающиеся значения. Опишите, какие особенности обнаружили."
- [x]  выполнено задание: "Какие факторы больше всего влияют на стоимость квартиры? Изучите, зависит ли цена от квадратного метра, числа комнат, этажа (первого или последнего), удалённости от центра. Также изучите зависимость от даты размещения: дня недели, месяца и года. "Выберите 10 населённых пунктов с наибольшим числом объявлений. Посчитайте среднюю цену квадратного метра в этих населённых пунктах. Выделите населённые пункты с самой высокой и низкой стоимостью жилья. Эти данные можно найти по имени в столбце '*locality_name'*. "
- [x]  выполнено задание: "Изучите предложения квартир: для каждой квартиры есть информация о расстоянии до центра. Выделите квартиры в Санкт-Петербурге (*'locality_name'*). Ваша задача — выяснить, какая область входит в центр. Создайте столбец с расстоянием до центра в километрах: округлите до целых значений. После этого посчитайте среднюю цену для каждого километра. Постройте график: он должен показывать, как цена зависит от удалённости от центра. Определите границу, где график сильно меняется — это и будет центральная зона. "
- [x]  выполнено задание: "Выделите сегмент квартир в центре. Проанализируйте эту территорию и изучите следующие параметры: площадь, цена, число комнат, высота потолков. Также выделите факторы, которые влияют на стоимость квартиры (число комнат, этаж, удалённость от центра, дата размещения объявления). Сделайте выводы. Отличаются ли они от общих выводов по всему городу?"
- [x]  в каждом этапе есть выводы
- [x]  есть общий вывод